In [1]:
import gmaps
import gmaps.datasets
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
import time

In [2]:
data = pd.read_csv('./acled_world_1997_2020.csv')
data.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7028992,887,YEM50795,50795,31 March 2020,2020,1,Explosions/Remote violence,Air/drone strike,Operation Restoring Hope,...,Jabal al Kanais,15.7696,45.4099,1,Yemen Press Agency; Yemen Data Project,Local partner-National,"On 31 March 2020, Saudi-led coalition air raid...",10,1586191627,YEM
1,7030784,76,BRA13328,13328,31 March 2020,2020,1,Strategic developments,Looting/property destruction,Military Forces of Brazil (2019-) Military Police,...,Foz do Iguacu,-25.5478,-54.5881,1,Portal Costa Oeste,Subnational,"On 31 March 2020, in Foz do Iguacu, Parana, mi...",0,1586210135,BRA
2,7028993,887,YEM50796,50796,31 March 2020,2020,1,Battles,Government regains territory,Military Forces of Yemen (2016-) Supreme Polit...,...,Jabal al Kanais,15.7696,45.4099,1,Yemen Data Project; Ofq News,Local partner-National,"On 31 March 2020, pro-Houthi forces claimed to...",0,1586191627,YEM
3,7028994,887,YEM50799,50799,31 March 2020,2020,1,Explosions/Remote violence,Air/drone strike,Operation Restoring Hope,...,Al Mazraq,16.5580,43.1981,1,Ansar Allah; Yemen Data Project,Local partner-National,"On 31 March 2020, four Saudi-led coalition air...",0,1586191627,YEM
4,7029250,887,YEM50785,50785,31 March 2020,2020,1,Battles,Armed clash,Military Forces of Yemen (2016-) Supreme Polit...,...,Wadi Yakla,14.7915,45.0387,2,Twitter,New media,"On 31 March 2020, clashes were reported betwee...",0,1586191628,YEM


In [20]:
class Animate(object):
    def __init__(self, data):
        self._data = data
        initial_year = min(self._data['year'])
        final_year = max(self._data['year'])
        self._heatmap=None
        self._slider = None
        
        title_widget = widgets.HTML(
        '<h3>Animation</h3>'
        '<h4>Data from <a href="https://www.acleddata.com">ACLED Project</h4>'
        )
        map_figure = self._render_map(initial_year)
        controls = self._render_controls(initial_year)
        self._container = widgets.VBox([title_widget, controls, map_figure])
        
    def render(self):
        display(self._container)
        
#     def start_animation(self):
#         while self._min_year<self._max_year:
#             self._min_year += 1
#             self._render_current_data(self._min_year)
#             time.sleep(3)

    def _on_year_change(self, change):
        year = self._slider.value
        self._heatmap.locations = self._render_current_data(year)
        return self._container
    
    def _render_map(self, initial_year):
        fig = gmaps.figure(map_type='ROADMAP')
        self._heatmap = gmaps.heatmap_layer(
        self._render_current_data(initial_year),
        max_intensity = 100,
        point_radius = 3
        )
        fig.add_layer(self._heatmap)
        return fig 
    
    def _render_controls(self, initial_year):
        self._slider = widgets.IntSlider(
            value=initial_year,
            min=min(self._data['year']),
            max=max(self._data['year']),
            description='Year',
            continuous_update=False
        )
        play = widgets.Play(
            interval = 8000,
            value=initial_year,
            min=min(self._data['year']),
            max=max(self._data['year']),
            step=1,
            disabled=False
        )
        widgets.jslink((self._slider,'value'),(play,'value'))
        self._slider.observe(self._on_year_change, names='value')
        controls = widgets.HBox([play, self._slider])
        return controls
    
    def _render_current_data(self, year):
        loc = self._data[self._data['year']==year]
        return loc[['latitude','longitude']]

In [22]:
animation = Animate(data)
animation.render()
# animation.start_animation()